In [1]:
!ulimit -n

8192


In [2]:
!pip install adversarial-robustness-toolbox

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier

import matplotlib.pyplot as plt
import time
import logging
import datetime
import random

cuda = torch.cuda.is_available()
cuda

True

In [4]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [5]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
      res = tensor + torch.randn(tensor.size()) * self.std + self.mean
      return torch.clamp(input=res, min=-0.5, max=0.5)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [6]:
class VGG16(nn.Module):
    def __init__(self, nb_classes=10):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16_bn()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.vgg16(x)
        x = self.linear(x)
        return x

In [7]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # loss.backward()
        # optimizer.step()

        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()

            # with torch.cuda.amp.autocast():
            #   outputs = model(data)

            #   _, predicted = torch.max(outputs.data, 1)
            #   total_predictions += target.size(0)
            #   correct_predictions += (predicted == target).sum().item()

            #   loss = criterion(outputs, target).detach()
            #   running_loss += loss.item()
            

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [8]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
file_hdlr = logging.FileHandler('/home/ubuntu/project/logs/'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

In [9]:
train_batchsize = 128
test_batchsize = 100
num_workers = 4
num_classes = 10

n_epochs = 30
img_size = 224
lr = 1e-4
min_lr = 1e-8
weight_decay = 5e-4
num_models = 10
noise_std = 0.06

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 'min_lr': min_lr, 'weight_decay': weight_decay, 'num_models': num_models, 'num_epochs': n_epochs, 'noise_std': noise_std}
logging.info(hyper_params)


2020-11-07 04:07:00,968 INFO {'lr': 0.0001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_models': 10, 'num_epochs': 30, 'noise_std': 0.06}


# Ensemble Class Definition

In [10]:
class MyEnsemble(nn.Module):
    def __init__(self, model_list, num_models_selected, num_classes=10):
        super(MyEnsemble, self).__init__()
        self.model_list = []
        self.num_models = len(model_list)
        self.num_models_selected = num_models_selected
        self.num_classes = num_classes
        for model in model_list:
          self.model_list.append(model)
        self.softmax = nn.Softmax()
        
    def forward(self, x, y):
        # generate random indices
        indices = [i for i in range(self.num_models)]
        random.shuffle(indices)
        indices = indices[:self.num_models_selected]
        models_selected = [self.model_list[idx] for idx in indices]

        noise = torch.randn(x.size()) * noise_std
        noise = noise.to(device)
        x_noised = x + noise

        loss_sum = torch.Tensor(0.0)
        
        for model in models_selected:
            model = model.to(device)
            logits = model(x_noised)
            loss = criterion(logits, y)
            loss_sum += loss
            model = model.to('cpu')
        
        return loss_sum
    
    def predict(self, x):
        raise NotImplemented("predict")

In [11]:
class MyPytorchClassifier(PyTorchClassifier):
    def loss_gradient(self, x: np.ndarray, y: np.ndarray, **kwargs) -> np.ndarray:
        """
        Compute the gradient of the loss function w.r.t. `x`.
        :param x: Sample input with shape as expected by the model.
        :param y: Target values (class labels) one-hot-encoded of shape `(nb_samples, nb_classes)` or indices of shape
                  `(nb_samples,)`.
        :return: Array of gradients of the same shape as `x`.
        """
        import torch  # lgtm [py/repeated-import]

        # Apply preprocessing
        x_preprocessed, y_preprocessed = self._apply_preprocessing(x, y, fit=False)

        # Check label shape
        y_preprocessed = self.reduce_labels(y_preprocessed)

        # Convert the inputs to Tensors
        inputs_t = torch.from_numpy(x_preprocessed).to(self._device)
        inputs_t.requires_grad = True

        # Convert the labels to Tensors
        labels_t = torch.from_numpy(y_preprocessed).to(self._device)

        # Compute the gradient and return
        model_outputs = self._model(inputs_t, labels_t)

        # Clean gradients
        self._model.zero_grad()

        # Compute gradients
        loss.backward()
        grads = inputs_t.grad.cpu().numpy().copy()  # type: ignore
        grads = self._apply_preprocessing_gradient(x, grads)
        assert grads.shape == x.shape

        return grads

In [12]:
# class MyEnsemble(nn.Module):
#     def __init__(self, model_list, num_models, num_models_selected, num_classes=10):
#         super(MyEnsemble, self).__init__()
#         self.model_list = []
#         self.num_models = num_models
#         self.num_models_selected = num_models_selected
#         self.num_classes = num_classes
#         for model in model_list:
#           self.model_list.append(model)
#         # Remove last linear layer
#         self.softmax = nn.Softmax()
        
#     def forward(self, x):
#         # mask = torch.randperm(num_models)[:num_models_selected]
#         !nvidia-smi
#         indices = [i for i in range(self.num_models)]
#         random.shuffle(indices)
#         indices = indices[:self.num_models_selected]
#         models_selected = [self.model_list[idx] for idx in indices]

#         logits_list = []
#         labels_list = []

#         noise = torch.randn(x.size()) * noise_std
#         noise = noise.to(device)
#         x_noised = x + noise

#         for model in models_selected:
#           logits = model(x_noised)
#           label_out = torch.unsqueeze(torch.argmax(self.softmax(logits), dim=1), dim=0)
#           logits = torch.unsqueeze(logits, dim=0)
#           labels_list.append(label_out)
#           logits_list.append(logits)
        
#         # majority vote
#         labels_tensor = torch.cat(labels_list, dim=0)
#         logits_tensor = torch.cat(logits_list, dim=0)
#         voted_class = torch.max(labels_tensor, dim=0) # possibly ties
#         mask = (labels_tensor == voted_class.values).int()

#         mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
#         mask = mask.transpose(1, 2)

#         sum_logits = torch.sum(mask * logits_tensor, dim=0)
#         divider = torch.sum(mask, dim=0)
        
#         del logits_list
#         del labels_list
#         del labels_tensor
#         del logits_tensor
#         del voted_class
#         del mask
#         models_selected = []
#         !nvidia-smi
#         return sum_logits / divider

#         # max_logits = torch.max(mask * logits_tensor, dim=0)
#         # return max_logits.values


In [13]:
# class MyEnsemble(nn.Module):
#     def __init__(self, model_list, num_models, num_models_selected, num_classes=10):
#         super(MyEnsemble, self).__init__()
#         self.model_list = []
#         self.num_models = num_models
#         self.num_models_selected = num_models_selected
#         self.num_classes = num_classes
#         for model in model_list:
#           self.model_list.append(model)
#         # Remove last linear layer
#         self.softmax = nn.Softmax()
        
#     def forward(self, x):
#         indices = [i for i in range(self.num_models)]
#         random.shuffle(indices)
#         indices = indices[:self.num_models_selected]
#         model_names_selected = [self.model_list[idx] for idx in indices]
        
#         models_selected = []
        
#         for model_name in model_names_selected:
#           model = VGG16()
#           model_data = torch.load('/home/ubuntu/project/{}'.format(model_name))
#           model.load_state_dict(model_data['model_state_dict'])
#           model.to(device)
#           models_selected.append(model)

#         logits_list = []
#         labels_list = []

#         noise = torch.randn(x.size()) * noise_std
#         noise = noise.to(device)
#         x_noised = x + noise

#         for model in models_selected:
#           logits = model(x_noised)
#           label_out = torch.unsqueeze(torch.argmax(self.softmax(logits), dim=1), dim=0)
#           logits = torch.unsqueeze(logits, dim=0)
#           labels_list.append(label_out)
#           logits_list.append(logits)
        
#         # majority vote
#         labels_tensor = torch.cat(labels_list, dim=0)
#         logits_tensor = torch.cat(logits_list, dim=0)
#         voted_class = torch.max(labels_tensor, dim=0) # possibly ties
#         mask = (labels_tensor == voted_class.values).int()

#         mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
#         mask = mask.transpose(1, 2)

#         sum_logits = torch.sum(mask * logits_tensor, dim=0)
#         divider = torch.sum(mask, dim=0)
        
#         for model in models_selected:
#             del model
#         del models_selected

#         return sum_logits / divider

#         # max_logits = torch.max(mask * logits_tensor, dim=0)
#         # return max_logits.values


# Ensemble Parameters

In [14]:
model_names = ['Model_30', 
               'Model_31',
               'Model_32',
               'Model_33',
               'Model_34',
               'Model_35',
               'Model_90',
               'Model_92',
               'Model_93',
               'Model_94',
               'Model_a',
               'Model_b',
               'mod_k_1',
               'mod_k_2',
               'mod_k_3',
               'mod_k_4']

In [15]:
model_names = model_names[:5]
num_models = len(model_names)
num_models_selected = 3
num_classes = 10
print(num_models)

5


# Load Saved Models

In [16]:
!nvidia-smi

Sat Nov  7 04:07:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
model_list = []
for model_name in model_names:
  model = VGG16()
  model_data = torch.load('/home/ubuntu/project/{}'.format(model_name))
  model.load_state_dict(model_data['model_state_dict'])
  model_list.append(model)

In [19]:
!nvidia-smi

Sat Nov  7 04:29:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |   3930MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Create Ensemble Object

In [20]:
my_ensemble = MyEnsemble(model_list, num_models_selected, num_classes=10)
# my_ensemble = MyEnsemble(model_names, num_models, num_models_selected, num_classes=10)
#my_ensemble.to(device)

# Create classifier object

In [21]:
criterion = nn.CrossEntropyLoss()
classifier = MyPytorchClassifier(
    model=my_ensemble,
    clip_values=(0, 1),
    loss=criterion,
    input_shape=(3, 224, 224),
    nb_classes=10,
)

2020-11-07 04:29:51,695 INFO Inferred 1 hidden layers on PyTorch classifier.


# Create attack object

In [22]:
attack = FastGradientMethod(estimator=classifier, eps=0.01)
# attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=0.1, eps_step=0.01)

# Get test examples

In [23]:
test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1))])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified


In [24]:
# x_test = []
# y_test = []
# for batch_idx, (X, Y) in enumerate(testloader):
#   x_test.append(X.squeeze().numpy())
#   y_test.append(Y[0].item())
# x_test = np.array(x_test)
# print(x_test.shape)

In [29]:
x_test = []
y_test = []
for (x, y) in testset:
    x_test.append(x.num)
    y_test.append(y)

10000


# Generating attack samples

In [23]:
!nvidia-smi

Thu Nov  5 22:56:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |   6058MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
x_test_adv = attack.generate(x=x_test[:100])

2020-11-05 22:56:55,000 INFO Using model predictions as correct labels for FGM.
/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: CUDA out of memory. Tried to allocate 196.00 MiB (GPU 0; 14.76 GiB total capacity; 13.77 GiB already allocated; 97.75 MiB free; 13.90 GiB reserved in total by PyTorch)

In [25]:
!nvidia-smi

Thu Nov  5 22:57:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |  15012MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
for i in range(len(y_test)):
  y_test[i] = y_test[i].item()

In [91]:
predictions = classifier.predict(x_test_adv)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [92]:
accuracy = np.sum(np.argmax(predictions, axis=1) == np.array(y_test)[:100]) / 100

In [93]:
accuracy

0.07

In [ ]:
model0 = VGG16()
model_data = torch.load('gdrive/My Drive/IDL_Project/Model_0')
model0.load_state_dict(model_data['model_state_dict'])
model0.to(device)

In [78]:
test_model(model=model0, test_loader=testloader, criterion=criterion)

2020-11-04 02:51:57,260 INFO Testing Loss: 0.3685084188319378
2020-11-04 02:51:57,261 INFO Testing Accuracy: 88.17%


(0.3685084188319378, 88.17)